In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
pd.options.mode.chained_assignment = None
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix
from tensorflow import keras
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import pickle

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
pip install -U emoji

     |████████████████████████████████| 51kB 1.4MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42176 sha256=d02c453688ad0ce92ec96109af565f6209ca383cbfb81f0f442c262226a0190e
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


In [0]:
exclude = set(string.punctuation) 
import re
def remove_punctuation(x): 
  """ Helper function to remove punctuation from a string x: any string """
  try: 
    x= re.sub(r"[-+]?[0-9]+(\.[0-9]+)?","",x)
    y = ''.join(ch for ch in x if ch not in exclude) 
  except:
    pass 
  return y 


In [0]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

In [0]:
# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(string):
    string=str(string)
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [0]:
import emoji
from nltk.corpus import stopwords

In [0]:
#STOPWORDS = set({'an', "you'll", "it's", 'each', 'at', 'ain', "mustn't", 'we', 'mightn', 'here', 'from', 'he', 'between', 'no', "haven't", 'to', 'how', "aren't", 'wouldn', 'didn', 'same', 'won', "should've", 'isn', 'both', 'did', 'weren', 'a', "wouldn't", 'ourselves', "you've", 'i', 'such', 'is', 'don', 'them', 'this', 'mustn', 'your', 'further', 'which', 'should', 'himself', 'over', 'most', 'out', 'does', 'am', 'those', 'by', "shouldn't", 'do', 'shouldn', 'be', 'there', "that'll", "won't", "shan't", 'own', "doesn't", 'ma', "isn't", 'all', 'then', 'she', 'hers', 'until', 'after', 'but', 've', 'our', 'if', 'about', 'can', 'her', 'these', 'again', "hasn't", 'nor', 'him', "you'd", 'where', 'with', "she's", 'on', 'of', 'other', 'very', 'ours', 'during', 't', 'had', 'because', "weren't", 'yourselves', 'than', "hadn't", 'me', 'doing', 'against', 'now', 'doesn', 'wasn', 'too', 'you', 'what', 'd', 're', "didn't", 'myself', 'his', "needn't", 'are', 'under', 'off', 'herself', 'up', 'some', 'itself', 'been', 'were', 'through', 'when', 'as', 'for', 'who', 'theirs', 'so', 'below', 'll', 'hadn', 'shan', 'any', 'its', 'and', 'in', "mightn't", 'before', 'while', 'that', 'has', 'yours', 'it', 'down', 'my', 'whom', 'their', 'few', "couldn't", 'hasn', 'why', "you're", 'or', 'haven', 'needn', 's', "don't", 'couldn', 'above', 'm', 'being', 'only', 'y', 'will', 'the', 'once', 'into', 'having', 'aren', 'more', 'themselves', 'have', 'just', 'o', 'was', "wasn't", 'they', 'not', 'yourself'})
#print(STOPWORDS)
#def remove_stopwords(text):
#    """custom function to remove the stopwords"""
#    return " ".join([word for word in str(text).split() if word not in STOPWORDS])



In [0]:
def preprocessing(subject,body):
  
  review= subject +" "+ body
  review= review.lower()
  review= remove_emoji(review)
  review = remove_punctuation(review)
  #review = remove_stopwords(review)
  review =stem_words(review)
  return review

In [13]:
from keras import backend as K
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall
def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)
    

Using TensorFlow backend.


In [14]:
model=keras.models.load_model("gdrive/My Drive/All reviews/ModelNoBugsFinal.h5", custom_objects={'f1': f1,'precision':precision,'recall':recall})

In [0]:
classes=['Advertising' ,'Battery'  ,'Camera & Photos' ,'Complexity',
 'Connectivity & HDMI', 'Customer Support', 'Design & UX' ,'Devices',
 'Feature Requests' ,'Frequency', 'Gaming', 'Import Export',
 'Internationalization', 'Location Services' ,'Notifications & Alerts',
 'Operating System' ,'Performance' ,'Pricing & Payment' ,'Privacy',
 'Security & Accounts', 'Sign Up & Login', 'Social & Collaboration',
 'Streaming & Video & Audio' ,'Update']

In [16]:
subject = input("Subject = ")
body= input ("Body = ")
review = preprocessing(subject=subject,body=body) 

Subject = the video is not working
Body = can someone help me with that 


In [17]:
print (review)

the video is not work can someon help me with that


In [0]:
transformer = TfidfTransformer()
loaded_vec = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("gdrive/My Drive/All reviews/feature.pkl", "rb")))
tfidfReview = transformer.fit_transform(loaded_vec.fit_transform(np.array([review])))

In [0]:
def predict (model,review): 
  mask = model.predict(tfidfReview.toarray())>0.3
  x=np.array(classes)[mask[0]]

  if len(x)==0 :
    if np.max(np.array(model.predict(tfidfReview.toarray()))) > 0.2 :
      return [classes[np.argmax(np.array(model.predict(tfidfReview.toarray())))]]
  return x

In [20]:
print(predict(model,tfidfReview))

['Customer Support' 'Streaming & Video & Audio']


In [21]:
print(model.predict(tfidfReview.toarray()))

[[8.9460611e-04 1.6686320e-04 1.3228655e-03 7.8725815e-04 1.4264047e-02
  9.8918104e-01 9.9847913e-03 1.8669665e-03 7.6290965e-04 9.0557337e-04
  4.2457283e-03 1.2107503e-04 1.6973913e-03 1.8116832e-04 1.3498366e-03
  2.1812320e-04 4.8539042e-03 1.7830729e-04 4.6828389e-04 3.5133140e-05
  1.3640314e-02 4.2149425e-04 9.9694687e-01 4.4902563e-03]]


In [22]:
mask = model.predict(tfidfReview.toarray())>0.3
print (np.array(classes)[mask[0]])

['Customer Support' 'Streaming & Video & Audio']
